# Лабораторная работа №8
## Генерация текста на основе “Алисы в стране чудес”
## Группа: БФИ1901

### Цель работы:
Создать генеративную модель для создания текста на основе "Алисы в стране чудес"

### Задание:
* Ознакомиться с генерацией текста
* Ознакомиться с системой Callback в Keras

### Ход работы:

#### Вопрос 1: Реализовать модель ИНС, которая будет генерировать текст
Ниже представлен полный код нашей программы, которая создает модель для генерации текста.

#### Импорт зависимостей
Начнем с импорта необходимых зависимостей для предварительной обработки данных и построения модели.

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

#### Получение и обработка данных
Затем нам нужно загрузить текст ASCII для книги в память и преобразовать все символы в нижний регистр, чтобы уменьшить словарный запас, который должна выучить сеть.

In [2]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Теперь, когда книга загружена, мы должны подготовить данные для моделирования нейронной сетью. Создадим набор всех отдельных символов в книге, а затем создадим карту каждого символа с уникальным целым числом.

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

Теперь, когда книга загружена и карта подготовлена, мы можем суммировать набор данных.

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144431
Total Vocab:  45


Разделяя книгу на эти последовательности, мы конвертируем символы в целые числа, используя нашу таблицу поиска, которую мы подготовили ранее.

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144331


Теперь, когда мы подготовили наши тренировочные данные, нам нужно преобразовать их так, чтобы они подходили для использования с Keras.

In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#### Настройка и обучение модели
Теперь мы можем определить нашу модель LSTM.

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Из-за медлительности данной модели и из-за наших требований по оптимизации мы будем использовать контрольные точки модели для записи всех сетевых весов, чтобы каждый раз регистрировать улучшение потерь в конце эпохи. Мы будем использовать лучший набор весов (наименьшая потеря), чтобы реализовать нашу генеративную модель в следующем разделе.

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:03d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Теперь осталось обучить модель.

In [9]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1128/1128 [==============================] - ETA: 0s - loss: 2.9786
Epoch 1: loss improved from inf to 2.97858, saving model to weights-improvement-001-2.9786.hdf5
1128/1128 [==============================] - 283s 249ms/step - loss: 2.9786
Epoch 2/20
1128/1128 [==============================] - ETA: 0s - loss: 2.7720
Epoch 2: loss improved from 2.97858 to 2.77205, saving model to weights-improvement-002-2.7720.hdf5
1128/1128 [==============================] - 286s 254ms/step - loss: 2.7720
Epoch 3/20
1128/1128 [==============================] - ETA: 0s - loss: 2.6677
Epoch 3: loss improved from 2.77205 to 2.66770, saving model to weights-improvement-003-2.6677.hdf5
1128/1128 [==============================] - 288s 255ms/step - loss: 2.6677
Epoch 4/20
1128/1128 [==============================] - ETA: 0s - loss: 2.5898
Epoch 4: loss improved from 2.66770 to 2.58985, saving model to weights-improvement-004-2.5898.hdf5
1128/1128 [==============================] - 287s 255ms/step

#### Генерация текста с помощью сети LSTM
Генерация текста с использованием обученной сети LSTM относительно проста.
Во-первых, мы загружаем данные и определяем сеть точно таким же образом, за исключением того, что веса сети загружаются из файла контрольных точек, и сеть не нуждается в обучении.

In [10]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "weights-improvement-20-1.9268.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Кроме того, при подготовке сопоставления уникальных символов с целыми числами мы также должны создать обратное отображение, которое мы можем использовать для преобразования целых чисел обратно в символы, чтобы мы могли понять предсказания.

In [11]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Простейший способ использования модели Keras LSTM для прогнозирования - сначала начать с последовательности начальных чисел в качестве входных данных, сгенерировать
следующий символ, затем обновить последовательность начальных чисел, чтобы добавить сгенерированный символ в конце, и обрезать первый символ. Этот процесс повторяется до тех пор, пока мы хотим предсказать новые символы (например, последовательность длиной 1000 символов).

In [12]:
import sys

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")


Seed:
" suddenly that alice had not a moment to think
about stopping herself before she found herself fallin "
g hor the sooe of the court, 'a didrter to make to tey the horse oo the soidt to tay in the cousd, and the was sointing to the white rabbit was an in was in the houph, and whst hn th the coumouse thin she was toe oade to the while sabbit was an the could so tee what she was not in the wag oo the had sooe th the court, 'and the more was o was to the woid th the soies so tay in the coutd ' 
'i moot than yhu ho a corno,s dreieed,' said the dotmouse, ''to the sare thing sas oo the sorpse ' she said to herself, and whin at the could not to the corrt, 
an cel to see the matter were toie at the could, 
'the donstes taa io the cortter thing to tou the wou se teln io to ' she said to herself, and whin a little so to the wan so the tooe of the tooe of the gourt, and ths go wo her to ce i vaed to toie that she was no the was oo the wan so the oooe tf the gorrt of the court, 'and the more

#### Вопрос 2: Написать собственный CallBack, который будет показывать то как генерируется текст во время обучения (то есть раз в какое-то количество эпох генирировать и выводить текст у необученной модели)

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import io

filename = "wonderland.txt"
text = open(filename).read().lower()
text = text.replace("\n", " ")

chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

seq_length = 40

n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

dataX = []
dataY = []

for i in range(0, len(text) - seq_length, 1):
    dataX.append(text[i : i + seq_length])
    dataY.append(text[i + seq_length])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

x = np.zeros((len(dataX), seq_length, len(chars)), dtype=bool)
y = np.zeros((len(dataX), len(chars)), dtype=bool)

for i, sentence in enumerate(dataX):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[dataY[i]]] = 1


model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam')

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 3 == 0:
            start_index = random.randint(0, len(text) - seq_length - 1)
            generated = ""
            sentence = text[start_index : start_index + seq_length]

            print ("\nSeed:")
            print (sentence)

            print ("Generated text:")

            for i in range(500):
                x_pred = np.zeros((1, seq_length, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_to_int[char]] = 1.0
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds)
                next_char = int_to_char[next_index]
                sentence = sentence[1:] + next_char
                generated += next_char

            print(generated)

model.fit(x, y, epochs=20, batch_size=512, callbacks=[CustomCallback()])

Total Characters:  144431
Total Vocab:  44
Total Patterns:  144391
Epoch 1/20
282/283 [============================>.] - ETA: 0s - loss: 2.7641
Seed:
. (as that is rather a hard word, i will
Generated text:
ktenhiwhiskwatintherans, fere  ofeekil. 'd  akk iee utannehwo 's inisos,t fucind oat ind lom,  alicrit th!lt iisrupgatler thethar, gor wait.  'lit talstgpratn" ye cou:lous olira.e dy shith. 'wettit, awe daaole-, aadsin tod  eicangopulef ind che owy eid.  we f wit ho geme heooo ty thran, say, sey ahtsemoa y in-malcy hufvwiosihing atsescalasee. t'inwhhanf atmelsy ny witlesiesdon  tffusin? isite heost, on.t te chevebing see the hil!-ead' oreisewan chs boke.   ahe, isit soeer.' aasle thiagat thin he
283/283 [==============================] - 117s 409ms/step - loss: 2.7641
Epoch 2/20
283/283 [==============================] - 93s 330ms/step - loss: 2.2468
Epoch 3/20
283/283 [==============================] - 94s 333ms/step - loss: 2.0588
Epoch 4/20
282/283 [============================>.

#### Вопрос 3: Отследить процесс обучения при помощи TensorFlowCallBack (TensorBoard), в отчете привести результаты и их анализ

In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import io
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

filename = "wonderland.txt"
text = open(filename).read().lower()
text = text.replace("\n", " ")

chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

seq_length = 40

n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

dataX = []
dataY = []

for i in range(0, len(text) - seq_length, 1):
    dataX.append(text[i : i + seq_length])
    dataY.append(text[i + seq_length])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

x = np.zeros((len(dataX), seq_length, len(chars)), dtype=bool)
y = np.zeros((len(dataX), len(chars)), dtype=bool)

for i, sentence in enumerate(dataX):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[dataY[i]]] = 1


model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam')

tb_callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0)

model.fit(x, y, epochs=20, batch_size=512, callbacks=[tb_callback])

Total Characters:  144431
Total Vocab:  44
Total Patterns:  144391
Epoch 1/20
283/283 [==============================] - 101s 351ms/step - loss: 2.7459
Epoch 2/20
283/283 [==============================] - 99s 351ms/step - loss: 2.2475
Epoch 3/20
283/283 [==============================] - 97s 344ms/step - loss: 2.0677
Epoch 4/20
283/283 [==============================] - 98s 345ms/step - loss: 1.9494
Epoch 5/20
283/283 [==============================] - 98s 345ms/step - loss: 1.8521
Epoch 6/20
283/283 [==============================] - 98s 345ms/step - loss: 1.7709
Epoch 7/20
283/283 [==============================] - 99s 349ms/step - loss: 1.6988
Epoch 8/20
283/283 [==============================] - 101s 357ms/step - loss: 1.6427
Epoch 9/20
283/283 [==============================] - 100s 353ms/step - loss: 1.5871
Epoch 10/20
283/283 [==============================] - 99s 349ms/step - loss: 1.5383
Epoch 11/20
283/283 [==============================] - 98s 346ms/step - loss: 1.4956
Epoc

Теперь запустим TensorBoard чтобы посмотреть на историю обучения нашей модели:

In [20]:
%load_ext tensorboard
%tensorboard --logdir logs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2476), started 4:00:41 ago. (Use '!kill 2476' to kill it.)

In [21]:
%reload_ext tensorboard

In [22]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2476), started 4:00:41 ago. (Use '!kill 2476' to kill it.)